### Методы классов

In [10]:
class Point:
    color = 'red'
    circle = 2

    # self - ссылка на экземпляр класса, позволяет работать с локальными атрибутами конкретного экземпляра
    def set_coords(self):
        print("Вызов метода set_coords" + str(self))

In [2]:
Point.set_coords

<function __main__.Point.set_coords()>

In [3]:
Point.set_coords()

Вызов метода set_coords


In [11]:
pt = Point()

In [12]:
pt.set_coords()

Вызов метода set_coords<__main__.Point object at 0x11078f5e0>


In [36]:
class Point:
    color = 'red'
    circle = 2
    # инициализатор класса
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    # финализатор класса
    def __del__(self):
        print("Удаление экземпляра:" + str(self))
        
    # self - ссылка на экземпляр класса
    def set_coords(self, x, y):
        self.x = x
        self.y = y

    def get_coords(self):
        return (self.x, self.y)

In [27]:
pt = Point()
pt.set_coords(1, 2)
print(pt.__dict__)

{'x': 1, 'y': 2}


In [28]:
pt2 = Point()
pt2.set_coords(10, 20)
print(pt2.__dict__)

{'x': 10, 'y': 20}


In [29]:
pt.get_coords()

(1, 2)

Методы также являются атрибутами классов

In [30]:
f = getattr(pt, 'get_coords')

In [31]:
f()

(1, 2)

In [34]:
new_pt = Point()
print(new_pt.__dict__)

{'x': 0, 'y': 0}
